### **Manejo de la paginación**

Trabajamos sobre el sitio:

https://www.audible.com/adblbestsellers?ref=a_search_t1_navTop_pl0cg1c0r0&pf_rd_p=adc4b13b-d074-4e1c-ac46-9f54aa53072b&pf_rd_r=1F7DV0MPHV77Z61RX566

Vemos que el enlace tiene 5 páginas:

<center><img src="https://i.postimg.cc/JzqJLpNt/ws-17.png"></center>

El tag `<ul>` nos indica toda la barra de paginación:

<center><img src="https://i.postimg.cc/QMd7xcmm/ws-18.png"></center>

Si scroleamos en el tag `<ul>` encontramos el tag `<li>` ue nos indican cada una de las páginas:

<center><img src="https://i.postimg.cc/Yq0TgP9B/ws-19.png"></center>

Para obtener la última página, utilizamos los indices en reversa:

<center><img src="https://i.postimg.cc/8z14MJwJ/ws-20.png"></center>

Para obtener el botón para avanzar la página, utilizamos el siguiente tag:

<center><img src="https://i.postimg.cc/QCMRx4Pm/ws-21.png"></center>

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

# Modo Headless
options = Options()  # Inicializar una instancia de la clase Options
options.headless = False  # False -> Modo Headless desactivado
# options.add_argument('window-size=1920x1080')  # Establece un tamaño de ventana grande, para que se muestren todos los datos

# El enlace del sitio web con el que trabajaremos tiene 5 páginas mientras que el original tiene 25. Prueba el código con cualquiera de ellas
# Se recomienda utilizar Chrome, pero podriamos utilizar Firefox, Safari, Edge, etc.
website = "https://www.audible.com/adblbestsellers?ref=a_search_t1_navTop_pl0cg1c0r0&pf_rd_p=adc4b13b-d074-4e1c-ac46-9f54aa53072b&pf_rd_r=1F7DV0MPHV77Z61RX566"
# website = "https://www.audible.com/search"
driver = webdriver.Chrome(executable_path = r'C:\Users\Alfonso\Downloads\Programas\chromedriver\chromedriver.exe', options = options)
driver.get(website)
driver.maximize_window()

# Paginación 1
pagination = driver.find_element_by_xpath('//ul[contains(@class, "pagingElements")]')  # localización de la barra de paginación
pages = pagination.find_elements_by_tag_name('li')  # localizar cada página mostrada en la barra de paginación
last_page = int(pages[-2].text)  # obteniendo la última página con indexación negativa (comienza desde donde termina el array)

book_title = []
book_author = []
book_length = []

# Paginación 2
current_page = 1   # esta es la página que el bot comienza a scrapear

# El bucle while de abajo funcionará hasta que el bot llegue a la última página del sitio web, entonces se interrumpirá (break)
while current_page <= last_page:
    time.sleep(2)  # dejar que la página se muestre correctamente
    container = driver.find_element_by_class_name('adbl-impression-container ')
    products = container.find_elements_by_xpath('.//li[contains(@class, "productListItem")]')
    # products = container.find_elements_by_xpath('./li')

    for product in products:
        book_title.append(product.find_element_by_xpath('.//h3[contains(@class, "bc-heading")]').text)
        book_author.append(product.find_element_by_xpath('.//li[contains(@class, "authorLabel")]').text)
        book_length.append(product.find_element_by_xpath('.//li[contains(@class, "runtimeLabel")]').text)

    current_page = current_page + 1  # incrementa la página_actual en 1 después de extraer los datos
    # Localizar el botón next_page y pulsar sobre él. Si el elemento no está en la página web, pasar a la siguiente iteración.
    try:
        next_page = driver.find_element_by_xpath('.//span[contains(@class , "nextButton")]')
        next_page.click()
    except:
        pass

driver.quit()

df_books = pd.DataFrame({'title': book_title, 'author': book_author, 'length': book_length})
df_books.to_csv('books_pagination.csv', index=False)

In [3]:
df_books

,title,author,length
0,1. The Women,By: Kristin Hannah,Length: 14 hrs and 57 mins
1,2. Atomic Habits,By: James Clear,Length: 5 hrs and 35 mins
2,3. The Teacher,By: Freida McFadden,Length: 9 hrs and 35 mins
3,4. First Lie Wins,By: Ashley Elston,Length: 9 hrs and 16 mins
4,5. Fourth Wing,By: Rebecca Yarros,Length: 20 hrs and 43 mins
...,...,...,...
94,95. The Serpent and the Wings of Night,By: Carissa Broadbent,Length: 15 hrs and 4 mins
95,96. The Way of Kings,By: Brandon Sanderson,Length: 45 hrs and 30 mins
96,97. Tempting Promises,By: Corinne Michaels,Length: 9 hrs and 32 mins
97,98. A Court of Thorns and Roses (Part 1 of 2) ...,By: Sarah J. Maas,Length: 6 hrs and 4 mins
